In [ ]:
# default_exp core

# doc_collection

> API details.

In [ ]:
#export
from nbdev.showdoc import *
import importlib
import os
import sys
import inspect
import json
import pkg_resources
import pkgutil
import pandas
import warnings
import pydoc
from pathlib import Path
from elasticsearch import Elasticsearch

In [ ]:
#export
def replace(list_, str1, str2):
    i = list_.index(str1)
    return list_[:i] + [str2] + list_[i + 1:]

In [ ]:
#export
def get_top_hundred():
    if not os.path.isfile('data/top.json'):
        os.system('curl -X GET "https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json" > data/top.json')

    f = open('data/top.json')
    data = json.load(f)

    return [dictionary['project'] for dictionary in data['rows'][:100]]

In [ ]:
#export 
def pip_top_hundred():
    lib_names = get_top_hundred()
    lib_names = replace(lib_names, 'msrest', 'msrestazure')
    for lib_name in lib_names:
        os.system('pip install ' + lib_name)

In [ ]:
#export
def val(key):
    return key[0] != '_' and key[-1] != '_'

In [ ]:
#export
def extr(module_name):
    names = []
    texts = []
    paths = []

    finded_names, finded_texts, finded_paths = recclass(module_name, module_name.__name__)
    names.extend(finded_names)
    texts.extend(finded_texts)
    paths.extend(finded_paths)

    if hasattr(module_name, '__path__'):
        file = module_name.__path__
    else:
        file = [inspect.getfile(module_name)]

    for importer, key, ispkg in pkgutil.iter_modules(file):
        if val(key) and ispkg and hasattr(module_name, key):
            if key != module_name.__name__ and fullname(getattr(module_name, key)).startswith(fullname(module_name)):
                finded_names, finded_texts, finded_paths = extr(getattr(module_name, key))
                names.extend(finded_names)
                texts.extend(finded_texts)
                paths.extend(finded_paths)

    return names, texts, paths

In [ ]:
#export
def recclass(cl, path):
    names = []
    texts = []
    paths = []

    names.append(cl)
    texts.append(pydoc.render_doc(cl, renderer=pydoc.TextDoc()))
    paths.append(path)

    for key_, value_ in inspect.getmembers(cl, inspect.isroutine):
        if val(key_) and hasattr(cl, key_):
            names.append(getattr(cl, key_))
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)

    for key_, value_ in inspect.getmembers(cl, inspect.isfunction):
        if val(key_) and hasattr(cl, key_):
            names.append(getattr(cl, key_))
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)

    for key_, value_ in inspect.getmembers(cl, inspect.isdatadescriptor):
        if val(key_) and hasattr(cl, key_):
            names.append(getattr(cl, key_))
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)

    for key_, value_ in inspect.getmembers(cl, inspect.isclass):
        if val(key_) and hasattr(cl, key_):
            if fullname(getattr(cl, key_)).startswith(fullname(cl)):
                finded_names, finded_texts, finded_paths = recclass(getattr(cl, key_), path + '.' + key_)
                names.extend(finded_names)
                texts.extend(finded_texts)
                paths.extend(finded_paths)

    return names, texts, paths

In [ ]:
#export
def fullname(o):

    if inspect.ismodule(o):
        return o.__name__

    if inspect.isfunction(o) or inspect.isclass(o) or inspect.ismethod(o):
        return o.__module__ + '.' + o.__name__

    if inspect.isroutine(o):
        if hasattr(o, '__name__'):
            return o.__class__.__module__ + '.' + o.__name__
        else:
            return o.__class__.__module__ + '.' + o.__class__.__name__

    if inspect.isdatadescriptor(o):
        if o.__class__.__name__ == 'property' and o.fget != None:
            if o.fget.__module__ != None:
                return o.fget.__module__ + '.' + o.fget.__name__
            else:
                return o.fget.__name__
        else:
            return o.__class__.__module__ + '.' + o.__class__.__name__

    return o.__class__.__name__

In [ ]:
#export
def extract():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    installed_packages = pkg_resources.working_set
    installed_packages_list = sorted(["%s" % i.key for i in installed_packages])

    if installed_packages_list.count('transformers'):
        installed_packages_list.remove('transformers')

    unsolved_package_names = ['beautifulsoup4', 'typing-extensions', 'pyyaml', 'argon2-cffi', 'jupyter-client', 'jupyter-core']
    unsolved_import_names = ['bs4', 'typing', 'yaml', 'argon2', 'jupyter_client', 'jupyter_core']

    for i in range(len(unsolved_package_names)): #libs which I cant do general way
        if installed_packages_list.count(unsolved_package_names[i]):
            installed_packages_list = replace(installed_packages_list, unsolved_package_names[i], unsolved_import_names[i])

    names = []
    texts = []
    paths = []

    c = 0
    for lib_name in installed_packages_list:
        try:
            importlib.import_module(lib_name)

            finded_names, finded_texts, finded_paths = extr(sys.modules[lib_name])
            names.extend(finded_names)
            texts.extend(finded_texts)
            paths.extend(finded_paths)

            print(len(names) - c, 'files from', lib_name, 'added')
            c = len(names)

        except Exception:
            try:
                metadata_dir = pkg_resources.get_distribution(lib_name).egg_info
                for name in open('%s/%s' % (metadata_dir, 'top_level.txt')).read().rstrip().split('\n'):
                    if name != '' and val(name):
                        importlib.import_module(name)

                        finded_names, finded_texts, finded_paths = extr(sys.modules[name])
                        names.extend(finded_names)
                        texts.extend(finded_texts)
                        paths.extend(finded_paths)

                        print(len(names) - c, 'files from', name, 'added to')
                        c = len(names)

            except Exception:
                print('--------------- error during ' + lib_name + ' documentation extracting')

    doc = pandas.DataFrame({'text': texts, 'path': paths})
    doc_wo_dupl = doc.drop_duplicates(subset=['text'])
    d = doc_wo_dupl.groupby('text').agg(paths = ('path', lambda x: list(x)), library = ('path', lambda x: list(x)[0].split('.')[0])).reset_index()
    return d

In [ ]:
#export
def es_add_bulk():

    d = extract()

    es = Elasticsearch('http://localhost:9200')

    request_body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "my_english_analyzer": {
                        "type": "standard",
                        "max_token_length": 5,
                        "stopwords": "_english_"
                    }
                }
            }
        }
    }

    if es.indices.exists(index="doc"):
        es.indices.delete(index='doc')

    es.indices.create(index='doc', body=request_body)

    k = ({
            "_index": "doc",
            "text": text,
            "paths": paths,
            "library": library
         } for text, paths, library in d)

    helpers.bulk(es, k)

    print("success!")